<h2>
 A company wants to hire new employees. The budget of the company for the salaries is $70000. The company's criteria for hiring are:
Keep hiring the senior with the smallest salary until you can not hire any more seniors.
Use the remaning budget to hire the junior with the smallest salary.
Keep hiring the junior with the smallest salary until you can not hire any more junior.
Write a SQL query to find the seniors and juniors hired under the mentioned criteria.
</h2>

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Employee-hiring').getOrCreate()

In [3]:
from pyspark.sql.functions import to_timestamp
data=[(1,'Junior',10000),(2,'Junior',15000),(3,'Junior',40000),(4,'Senior',16000),(5,'Senior',20000),(6,'Senior',50000)
     ]

schema="""
 emp_id int,
experience string,
salary int
"""
df=spark.createDataFrame(data,schema)
df.show()

+------+----------+------+
|emp_id|experience|salary|
+------+----------+------+
|     1|    Junior| 10000|
|     2|    Junior| 15000|
|     3|    Junior| 40000|
|     4|    Senior| 16000|
|     5|    Senior| 20000|
|     6|    Senior| 50000|
+------+----------+------+



In [20]:
from pyspark.sql.functions import col,sum
from pyspark.sql.window import Window

window_spec= Window.partitionBy(col('experience')).orderBy(col('salary').asc())

senior_df=df.withColumn('cum_sal',sum(col('salary')).over(window_spec)).\
filter((col('cum_sal')<=70000) & (col('experience')=='Senior'))
senior_df.show()


+------+----------+------+-------+
|emp_id|experience|salary|cum_sal|
+------+----------+------+-------+
|     4|    Senior| 16000|  16000|
|     5|    Senior| 20000|  36000|
+------+----------+------+-------+



In [26]:
budget_left=70000-senior_df.select(col('cum_sal')).orderBy(col('cum_sal').desc()).limit(1).\
collect()[0]['cum_sal']

In [27]:
junior_df=df.withColumn('cum_sal',sum(col('salary')).over(window_spec)).\
filter((col('cum_sal')<=budget_left) & (col('experience')=='Junior'))
junior_df.show()

+------+----------+------+-------+
|emp_id|experience|salary|cum_sal|
+------+----------+------+-------+
|     1|    Junior| 10000|  10000|
|     2|    Junior| 15000|  25000|
+------+----------+------+-------+



In [31]:
result_df=senior_df.select(col('emp_id'),col('experience'),col('salary')).\
union(junior_df.select(col('emp_id'),col('experience'),col('salary'))).orderBy(col('emp_id').asc())
result_df.show()

+------+----------+------+
|emp_id|experience|salary|
+------+----------+------+
|     1|    Junior| 10000|
|     2|    Junior| 15000|
|     4|    Senior| 16000|
|     5|    Senior| 20000|
+------+----------+------+

